In [1]:
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import $file.^.cpinit, cpinit._
import ss.implicits._
import com.relatedsciences.opentargets.etl.pipeline.Functions
import org.apache.spark.sql.functions._
import java.nio.file.Paths
import org.apache.spark.sql.DataFrame
import org.apache.spark.storage.StorageLevel
import org.apache.spark.sql.expressions.Window

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/17 23:58:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $file.$          , sparkinit._

import $file.$         , pathinit._

import $file.$       , cpinit._

import ss.implicits._

import com.relatedsciences.opentargets.etl.pipeline.Functions

import org.apache.spark.sql.functions._

import java.nio.file.Paths

import org.apache.spark.sql.DataFrame

import org.apache.spark.storage.StorageLevel

import org.apache.spark.sql.expressions.Window

In [2]:
def prefix(prefix: String)(df: DataFrame): DataFrame = {
  val cols = df.columns.map(c => df(c).as(s"$prefix$c"))
  df.select(cols: _*)
}

defined function prefix

### Load Results (from v1 and v2 pipelines)

In [3]:
val dfv2 = ss.read.parquet(RESULTS_DIR.resolve("evidence_raw.parquet").toString)
    // TODO: remove this once the pipeline has been rerun
    .transform(Functions.addEvidenceRecordId("id"))

parquet at cmd2.sc:1

1 / 1

19/12/17 23:58:58 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


dfv2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [access_level: string, disease: struct<biosample: struct<id: string, name: string>, id: string ... 2 more fields> ... 11 more fields]

In [4]:
val dfv1 = ss.read.json(EXTRACT_DIR.resolve("evidence.json").toString)
    .transform(Functions.addEvidenceRecordId("id"))

json at cmd3.sc:1

115 / 115

dfv1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [access_level: string, disease: struct<biosample: struct<id: string, name: string>, efo_info: struct<efo_id: string, label: string ... 2 more fields> ... 3 more fields> ... 12 more fields]

In [5]:
dfv1.count

count at cmd4.sc:1

115 / 115

count at cmd4.sc:1

1 / 1

res4: Long = 1678005L

In [5]:
dfv2.count

count at cmd4.sc:1

24 / 24

count at cmd4.sc:1

1 / 1

res4: Long = 9447082L

In [24]:
assert(dfv1.select("id").distinct.count == dfv1.count)

count at cmd23.sc:1

115 / 115

count at cmd23.sc:1

1 / 1

count at cmd23.sc:1

115 / 115

count at cmd23.sc:1

1 / 1

In [6]:
dfv2.groupBy("id").count.groupBy("count").count.show

show at cmd5.sc:1

24 / 24

show at cmd5.sc:1

1 / 1

show at cmd5.sc:1

1 / 1

+-----+-------+
|count|  count|
+-----+-------+
|    2| 863400|
|    1|7681353|
|    5|    387|
|    3|   6677|
|    4|   3672|
|    8|     38|
|    6|    236|
|    7|     42|
|   12|      2|
|   11|      2|
|   10|      3|
|    9|     13|
|   15|      1|
|   13|      1|
|   22|      1|
|   18|      1|
+-----+-------+



### Validate Record Presence

Before equating values, simply check that the record ids are present in both datasets:

In [7]:
val df1 = dfv1.select($"id", $"sourceID", $"target.id".as("target_id"), $"disease.id".as("disease_id"))
    .transform(prefix("v1:")).dropDuplicates()
val df2 = dfv2.select($"id", $"sourceID", $"target.id".as("target_id"), $"disease.id".as("disease_id"))
    .transform(prefix("v2:")).dropDuplicates()
val dfj = df1.join(df2, df1("v1:id") === df2("v2:id"), "outer")
    .withColumn("status", 
        when($"v1:id".isNull && $"v2:id".isNotNull, "v2_only")
        .when($"v1:id".isNotNull && $"v2:id".isNull, "v1_only")
        .when($"v1:id".isNotNull && $"v2:id".isNotNull, "both")
        .otherwise("neither")
    )

df1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [v1:id: string, v1:sourceID: string ... 2 more fields]
df2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [v2:id: string, v2:sourceID: string ... 2 more fields]
dfj: DataFrame = [v1:id: string, v1:sourceID: string ... 7 more fields]

In [8]:
dfj.groupBy($"v2:sourceID", $"status").count.show

show at cmd7.sc:1

115 / 115

show at cmd7.sc:1

24 / 24

show at cmd7.sc:1

...

show at cmd7.sc:1

1 / 1

show at cmd7.sc:1

1 / 1

+------------------+-------+-------+
|       v2:sourceID| status|  count|
+------------------+-------+-------+
|           uniprot|   both|  31555|
|         europepmc|v2_only|6913511|
|cancer_gene_census|   both|  96274|
|        slapenrich|   both|  74570|
|  expression_atlas|   both| 202648|
|            chembl|   both| 384754|
|         phenodigm|   both| 501788|
|      gwas_catalog|   both| 186237|
|               eva|   both| 103019|
|       eva_somatic|   both|   7879|
|          reactome|   both|  10989|
|    phewas_catalog|   both|  61947|
|  genomics_england|   both|  10550|
|    gene2phenotype|   both|   1586|
|uniprot_literature|   both|   4818|
|           intogen|   both|   2375|
|            crispr|   both|   1844|
|      gwas_catalog|v2_only|    484|
|   uniprot_somatic|   both|    284|
|            sysbio|   both|    408|
+------------------+-------+-------+
only showing top 20 rows



Of the two discrepancies above, the EPMC differences are expected since more data was processed in the v2 pipeline.  The GWAS catalog data however is a result of the UniProt -> Non-reference gene -> reference Ensembl id translation being broken in the v1 pipeline.  Prove the latter is true:

In [9]:
val dfx = dfj.filter($"v2:sourceID" === "gwas_catalog" && $"status" === "v2_only").persist()
def idx = dfx.select("v2:id").collect.map(_(0).asInstanceOf[String])
dfv2.filter($"id".isin(idx:_*)).groupBy("sourceID", "type").count.show

collect at cmd8.sc:2

24 / 24

collect at cmd8.sc:2

115 / 115

collect at cmd8.sc:2

...

24 / 24

show at cmd8.sc:3

1 / 1

+------------+-------------------+-----+
|    sourceID|               type|count|
+------------+-------------------+-----+
|gwas_catalog|genetic_association|  526|
+------------+-------------------+-----+



dfx: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [v1:id: string, v1:sourceID: string ... 7 more fields]
defined function idx

In [10]:
// Load the non-reference gene mapping
val dfnr = ss.read
      .option("header", "true")
      .csv(MAIN_RESOURCE_DIR.resolve("data").resolve("genes_with_non_reference_ensembl_ids_lkp.tsv").toString)
dfnr.show(3, false)

csv at cmd9.sc:3

1 / 1

show at cmd9.sc:4

1 / 1

+-----------+---------------+---------------+
|gene_symbol|reference      |alternate      |
+-----------+---------------+---------------+
|ABCB11     |ENSG00000073734|ENSG00000276582|
|AGER       |ENSG00000204305|ENSG00000237405|
|AGK        |ENSG00000006530|ENSG00000262327|
+-----------+---------------+---------------+
only showing top 3 rows



dfnr: DataFrame = [gene_symbol: string, reference: string ... 1 more field]

In [11]:
// Show that of the v2 gwas_catalog records not in the v1 results, they all resolve to 
// reference genes in the non-reference mapping:
dfv2
    .filter($"id".isin(idx:_*))
    .withColumn("has_nonref_id", $"target.id".isin(
        dfnr.select("reference").collect.map(_(0).asInstanceOf[String]):_*
    ))
    .groupBy("has_nonref_id").count.show

collect at cmd8.sc:2

1 / 1

collect at cmd10.sc:4

1 / 1

show at cmd10.sc:6

24 / 24

show at cmd10.sc:6

1 / 1

+-------------+-----+
|has_nonref_id|count|
+-------------+-----+
|         true|  526|
+-------------+-----+



### Duplicate Record Validation

In [12]:
val df1 = dfv1.select("id")
val df2 = dfv2.select("id", "sourceID", "type")
// Find the ids that exist in both datasets but are non-unique in the v2 data
def dupeIds = dfv1.select("id").join(
        dfv2.select("id", "sourceID", "type"), 
        Seq("id"), 
        "inner"
    ).groupBy("sourceID", "type", "id").count.filter($"count" > 1)
    .select("id").collect.map(_(0).asInstanceOf[String])

// For the non-unique ids, count how often the same target + disease pair occurs
// for the sake of determining if the records are really different or not
val dfidct = dfv2
    .filter($"id".isin(dupeIds:_*))
    .withColumn("target_disease", concat($"target.id", lit(":"), $"disease.id"))
    .groupBy("sourceID", "type", "id")
    .agg(countDistinct("target_disease").as("ct_tgt_dis"), count("id").as("ct"))
    .cache()

collect at cmd11.sc:9

24 / 24

collect at cmd11.sc:9

115 / 115

collect at cmd11.sc:9

1 / 1

df1: DataFrame = [id: string]
df2: DataFrame = [id: string, sourceID: string ... 1 more field]
defined function dupeIds
dfidct: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [sourceID: string, type: string ... 3 more fields]

In [13]:
// Show data sources/types where the target + disease combinations for a duplicated id
// are all unique (indicating that they are almost certainly capturing extra information)
dfidct.filter($"ct_tgt_dis" === $"ct").groupBy("sourceID", "type")
    .agg(count("id").as("ctid"), sum("ct").as("ctrec")).show

show at cmd12.sc:2

24 / 24

show at cmd12.sc:2

1 / 1

show at cmd12.sc:2

1 / 1

show at cmd12.sc:2

1 / 1

+----------------+-------------------+----+-----+
|        sourceID|               type|ctid|ctrec|
+----------------+-------------------+----+-----+
|expression_atlas|     rna_expression| 136|  272|
|  phewas_catalog|genetic_association|4735|10405|
+----------------+-------------------+----+-----+



In [14]:
dfidct
    .filter($"ct_tgt_dis" === $"ct")
    .withColumn("rid", row_number.over(Window.partitionBy($"sourceID", $"type").orderBy($"ct_tgt_dis".desc)))
    .filter($"rid" <= 10)
    .show(20, false)

show at cmd13.sc:5

1 / 1

show at cmd13.sc:5

1 / 1

+----------------+-------------------+--------------------------------+----------+---+---+
|sourceID        |type               |id                              |ct_tgt_dis|ct |rid|
+----------------+-------------------+--------------------------------+----------+---+---+
|expression_atlas|rna_expression     |975717f23fabf725ae9428d3fd6288ca|2         |2  |1  |
|expression_atlas|rna_expression     |190e566aa22cd366bba1df7c5ace79a1|2         |2  |2  |
|expression_atlas|rna_expression     |a14c530e035d7ceb10f258551594ed5b|2         |2  |3  |
|expression_atlas|rna_expression     |72ed48343a39255b956add2ad5c5c7eb|2         |2  |4  |
|expression_atlas|rna_expression     |e1b5b5affdcdf36b8e3666e689b74e6a|2         |2  |5  |
|expression_atlas|rna_expression     |dab3d2e2f7430b49fc31cad8b8ff3e8d|2         |2  |6  |
|expression_atlas|rna_expression     |541ece3c99a3e74b90ed4408ffeb28de|2         |2  |7  |
|expression_atlas|rna_expression     |210399f7850d19fa6607d9fa888db051|2         |2  |8  |

#### Expression Atlas Issues

In [15]:
// Verify that All of the expression_atlas records correspond to one single issue
// with a study of Orphanet_586 and EFO_1000024
def geteaids = dfidct
    .filter($"ct_tgt_dis" === $"ct")
    .filter($"sourceID" === "expression_atlas")
    .select("id").distinct.collect.map(_.getString(0))
// This should be empty if there are any other EA issues
dfv2
    .filter($"id".isin(geteaids:_*))
    .filter(!$"disease.id".contains("Orphanet_586") && !$"disease.id".contains("EFO_1000024"))
    .select(
        $"id",
        $"target.id".as("target.id"),
        $"disease.id".as("disease.id"),
        $"sourceID".as("sourceID"),
        $"type".as("type"),
        to_json($"unique_association_fields")
    )
    .show(10, false)

collect at cmd14.sc:4

1 / 1

collect at cmd14.sc:4

1 / 1

show at cmd14.sc:17

...

4 / 4

show at cmd14.sc:17

19 / 19

+---+---------+----------+--------+----+----------------------------------------+
|id |target.id|disease.id|sourceID|type|structstojson(unique_association_fields)|
+---+---------+----------+--------+----+----------------------------------------+
+---+---------+----------+--------+----+----------------------------------------+



defined function geteaids

In [16]:
dfv2
    .filter($"sourceID" === "expression_atlas")
    .filter($"type" === "rna_expression")
    .filter($"target.id".contains("ENSG00000091137"))
    .filter($"disease.id".contains("Orphanet_586") || $"disease.id".contains("EFO_1000024"))
    .select(
        $"id",
        $"target.id",
        $"disease.id",
        to_json($"unique_association_fields")
    )
    .show(10, false)

show at cmd15.sc:12

1 / 1

show at cmd15.sc:12

4 / 4

show at cmd15.sc:12

19 / 19

+--------------------------------+---------------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id                              |id             |id          |structstojson(unique_association_fields)                                                                                                                                                                                                                   |
+--------------------------------+---------------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|265f3ff9391061134e8d116c23f16aca|ENSG00000091137|EF

#### PheWAS Catalog Issues

In [17]:
dfv2
    .filter($"id".isin("46ecc60c84b1e31974ac6908cc4f8f65"))
    .select(
        $"id",
        $"target.id".as("target.id"),
        $"disease.id".as("disease.id"),
        $"sourceID".as("sourceID"),
        $"type".as("type"),
        to_json($"unique_association_fields")
    )
    .show(10, false)

show at cmd16.sc:11

1 / 1

show at cmd16.sc:11

4 / 4

show at cmd16.sc:11

19 / 19

+--------------------------------+---------------+-----------+--------------+-------------------+------------------------------------------------------------------------------------------------------------------------+
|id                              |target.id      |disease.id |sourceID      |type               |structstojson(unique_association_fields)                                                                                |
+--------------------------------+---------------+-----------+--------------+-------------------+------------------------------------------------------------------------------------------------------------------------+
|46ecc60c84b1e31974ac6908cc4f8f65|ENSG00000139155|EFO_0004269|phewas_catalog|genetic_association|{"cases":"1338","odds_ratio":"1.149","phenotype":"Abnormal electrocardiogram [ECG] [EKG]","datasource":"phewas_catalog"}|
|46ecc60c84b1e31974ac6908cc4f8f65|ENSG00000130204|EFO_0004269|phewas_catalog|genetic_association|{"cases":"1338","odds_ratio

In [18]:
dfv1
    .filter($"id" === "46ecc60c84b1e31974ac6908cc4f8f65")
    .select(
        $"id",
        $"target.id".as("target.id"),
        $"disease.id".as("disease.id"),
        $"sourceID".as("sourceID"),
        $"type".as("type"),
        to_json($"unique_association_fields")
    )
    .show(10, false)

show at cmd17.sc:11

1 / 1

show at cmd17.sc:11

4 / 4

show at cmd17.sc:11

20 / 20

show at cmd17.sc:11

90 / 90

+--------------------------------+---------------+-----------+--------------+-------------------+------------------------------------------------------------------------------------------------------------------------+
|id                              |target.id      |disease.id |sourceID      |type               |structstojson(unique_association_fields)                                                                                |
+--------------------------------+---------------+-----------+--------------+-------------------+------------------------------------------------------------------------------------------------------------------------+
|46ecc60c84b1e31974ac6908cc4f8f65|ENSG00000122641|EFO_0004269|phewas_catalog|genetic_association|{"cases":"1338","odds_ratio":"1.149","phenotype":"Abnormal electrocardiogram [ECG] [EKG]","datasource":"phewas_catalog"}|
+--------------------------------+---------------+-----------+--------------+-------------------+---------------------------

### Field Validation

Fields to validate:
- evidence.gene2variant.resource_score.{type, value}
- evidence.resource_score.{type, value}
- evidence.evidence_codes (array of string)
- target.id
- disease.id

In [19]:
val cols = Seq(
    col("id"),
    col("sourceID"),
    col("evidence.gene2variant.resource_score.type").as("g2vrs_type"),
    col("evidence.gene2variant.resource_score.value").as("g2vrs_value"),
    col("evidence.resource_score.type").as("ers_type"),
    col("evidence.resource_score.value").as("ers_value"),
    sort_array(col("evidence.evidence_codes")).as("evidence_codes"),
    col("target.id").as("target_id"),
    col("disease.id").as("disease_id")
)
val df1 = dfv1.select(cols:_*).transform(prefix("v1:")).dropDuplicates("v1:id")
val df2 = dfv2.select(cols:_*).transform(prefix("v2:")).dropDuplicates("v2:id")
val dfm = df1.join(df2, df1("v1:id") === df2("v2:id"), "inner").persist()

cols: Seq[org.apache.spark.sql.Column] = List(
  id,
  sourceID,
  evidence.gene2variant.resource_score.type AS `g2vrs_type`,
  evidence.gene2variant.resource_score.value AS `g2vrs_value`,
  evidence.resource_score.type AS `ers_type`,
  evidence.resource_score.value AS `ers_value`,
  sort_array(evidence.evidence_codes, true) AS `evidence_codes`,
  target.id AS `target_id`,
  disease.id AS `disease_id`
)
df1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [v1:id: string, v1:sourceID: string ... 7 more fields]
df2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [v2:id: string, v2:sourceID: string ... 7 more fields]
dfm: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [v1:id: string, v1:sourceID: string ... 16 more fields]

In [20]:
val dfsct = dfm.columns.map(_.split(":").last).toSet.toSeq.sortWith(_ < _).map { c => 
    dfm
        .withColumn("source_id", col("v1:sourceID"))
        .withColumn("field", lit(c))
        .withColumn("status", 
            when(col("v1:" + c).isNull && col("v2:" + c).isNull, "both_null")
            .when(col("v1:" + c).isNotNull && col("v2:" + c).isNull, "v2_null")
            .when(col("v1:" + c).isNull && col("v2:" + c).isNotNull, "v1_null")
            .otherwise("not_null")
        )
        .withColumn("equal", 
            col("status") === "both_null" || 
            (col("status") === "not_null" && col("v1:" + c) === col("v2:" + c))
        )
        .groupBy("source_id", "field", "status", "equal").count
}.reduce(_.union(_))

dfsct: DataFrame = [source_id: string, field: string ... 3 more fields]

In [21]:
dfsct.groupBy("field", "status", "equal").sum("count").show

show at cmd20.sc:1

115 / 115

show at cmd20.sc:1

24 / 24

show at cmd20.sc:1

...

show at cmd20.sc:1

9 / 9

show at cmd20.sc:1

1 / 1

+--------------+---------+-----+----------+
|         field|   status|equal|sum(count)|
+--------------+---------+-----+----------+
|    disease_id| not_null| true|   1677916|
|    disease_id| not_null|false|        89|
|      ers_type|both_null| true|   1263608|
|      ers_type| not_null| true|    414397|
|     ers_value|both_null| true|   1263608|
|     ers_value| not_null| true|    414395|
|     ers_value| not_null|false|         2|
|evidence_codes| not_null| true|    791463|
|evidence_codes| not_null|false|    886542|
|    g2vrs_type| not_null| true|    377066|
|    g2vrs_type|both_null| true|   1300939|
|   g2vrs_value| not_null| true|    377066|
|   g2vrs_value|both_null| true|   1300939|
|            id| not_null| true|   1678005|
|      sourceID| not_null| true|   1678005|
|     target_id| not_null| true|   1673374|
|     target_id| not_null|false|      4631|
+--------------+---------+-----+----------+



In [22]:
dfsct.filter(!$"equal").orderBy("field", "source_id", "equal", "status").show(40)

show at cmd21.sc:1

1 / 1

show at cmd21.sc:1

1 / 1

show at cmd21.sc:1

...

show at cmd21.sc:1

1 / 1

show at cmd21.sc:1

9 / 9

+----------------+--------------+--------+-----+------+
|       source_id|         field|  status|equal| count|
+----------------+--------------+--------+-----+------+
|expression_atlas|    disease_id|not_null|false|    89|
|         intogen|     ers_value|not_null|false|     1|
|          sysbio|     ers_value|not_null|false|     1|
|          chembl|evidence_codes|not_null|false|384754|
|       phenodigm|evidence_codes|not_null|false|501788|
|  phewas_catalog|     target_id|not_null|false|  4631|
+----------------+--------------+--------+-----+------+



In [28]:
dfm
    .filter($"v1:sourceID" === "chembl")
    .filter($"v1:evidence_codes" =!= $"v2:evidence_codes")
    .select("v1:id", "v1:evidence_codes", "v2:evidence_codes")
    .show(10, false)

show at cmd27.sc:5

1 / 1

+--------------------------------+----------------------------------------+-----------------+
|v1:id                           |v1:evidence_codes                       |v2:evidence_codes|
+--------------------------------+----------------------------------------+-----------------+
|000020703b3b49983f5d526491ef410a|[ECO_0000205, drug_disease, target_drug]|[]               |
|0000539293103d4724c62b95c003958c|[ECO_0000205, drug_disease, target_drug]|[]               |
|00007c220c8ed6817903f807a15df1d8|[ECO_0000205, drug_disease, target_drug]|[]               |
|0000ad1bf6ecd8921e92eaab1951c036|[ECO_0000205, drug_disease, target_drug]|[]               |
|0000dbe78e5eab766324704f29c56004|[ECO_0000205, drug_disease, target_drug]|[]               |
|000140ed74c2b61b983cc4ed986775c1|[ECO_0000205, drug_disease, target_drug]|[]               |
|000142f8b39831464d1f6d8207c20422|[ECO_0000205, drug_disease, target_drug]|[]               |
|00015b2fa7069c36e9cbfb3888d57f91|[ECO_0000205, drug_disease

In [29]:
dfm
    .filter($"v1:sourceID".isin("sysbio", "intogen"))
    .filter($"v1:ers_value" =!= $"v2:ers_value")
    .select("v1:id", "v1:ers_value", "v2:ers_value")
    .show(10, false)

show at cmd28.sc:5

1 / 1

+--------------------------------+------------+------------+
|v1:id                           |v1:ers_value|v2:ers_value|
+--------------------------------+------------+------------+
|0823619202cd3bc34f24e18d0ae5a18f|0.859       |1.0         |
|ab8960325797b46011a721b4a2685868|0.5         |0.25        |
+--------------------------------+------------+------------+



In [26]:
dfm
    .filter($"v1:sourceID" === "phewas_catalog")
    .filter($"v1:target_id" =!= $"v2:target_id")
    .select("v1:id", "v1:target_id", "v2:target_id")
    .show(10, false)

show at cmd25.sc:5

1 / 1

+--------------------------------+---------------+---------------+
|v1:id                           |v1:target_id   |v2:target_id   |
+--------------------------------+---------------+---------------+
|0001c78e77ccaa1f2b43c7ab238ee209|ENSG00000116675|ENSG00000135966|
|0008f59c44bcc858cd6e50934dc03963|ENSG00000135541|ENSG00000153930|
|0015a22619ee747ddbdb97d4b08ff4ed|ENSG00000131183|ENSG00000142655|
|00190c742e0c96727c766b171c330dfe|ENSG00000157823|ENSG00000115604|
|002174bcd7d46dc170b7ed174df68b5b|ENSG00000139926|ENSG00000138193|
|004031f19e8c0a7651dcd3c5444f4858|ENSG00000010404|ENSG00000214548|
|00485e26a5d1c1e21be763a9ed20b070|ENSG00000197891|ENSG00000258839|
|00625928661ba5673e500fec75a2f020|ENSG00000125851|ENSG00000179344|
|00870f4a20497da117462edbebbee0bf|ENSG00000134538|ENSG00000145777|
|009318f52f93e31cca73c0630850d5be|ENSG00000164379|ENSG00000167513|
+--------------------------------+---------------+---------------+
only showing top 10 rows



### Raw Record Search

In [33]:
dfm
    .filter($"v1:sourceID" === "chembl")
    .filter($"v1:evidence_codes" =!= $"v2:evidence_codes")
    .select("v1:target_id", "v1:disease_id", "v1:id", "v1:evidence_codes", "v2:evidence_codes")
    .show(3, false)

show at cmd32.sc:5

1 / 1

+---------------+-------------+--------------------------------+----------------------------------------+-----------------+
|v1:target_id   |v1:disease_id|v1:id                           |v1:evidence_codes                       |v2:evidence_codes|
+---------------+-------------+--------------------------------+----------------------------------------+-----------------+
|ENSG00000159640|EFO_0000537  |000020703b3b49983f5d526491ef410a|[ECO_0000205, drug_disease, target_drug]|[]               |
|ENSG00000089558|EFO_0003911  |0000539293103d4724c62b95c003958c|[ECO_0000205, drug_disease, target_drug]|[]               |
|ENSG00000163517|EFO_0000182  |00007c220c8ed6817903f807a15df1d8|[ECO_0000205, drug_disease, target_drug]|[]               |
+---------------+-------------+--------------------------------+----------------------------------------+-----------------+
only showing top 3 rows



In [34]:
ss.read.json(EXTRACT_DIR.resolve("evidence_raw.json").toString)
    .filter($"sourceID" === "chembl")
    .filter($"target.id".contains("ENSG00000089558"))
    .filter($"disease.id".contains("EFO_0003911"))
    .select(
        "target.id",
        "disease.id",
        "evidence.evidence_codes",
        "evidence.variant2disease.evidence_codes",
        "evidence.gene2variant.evidence_codes",
        "evidence.target2drug.evidence_codes",
        "evidence.drug2clinic.evidence_codes",
        "evidence.biological_model.evidence_codes",
        "evidence.gene2variant.functional_consequence"
    )
    .show(10, false)

json at cmd33.sc:1

178 / 178

show at cmd33.sc:16

1 / 1

show at cmd33.sc:16

...

show at cmd33.sc:16

100 / 100

show at cmd33.sc:16

53 / 53

+---+---+--------------+--------------+--------------+--------------+--------------+--------------+----------------------+
|id |id |evidence_codes|evidence_codes|evidence_codes|evidence_codes|evidence_codes|evidence_codes|functional_consequence|
+---+---+--------------+--------------+--------------+--------------+--------------+--------------+----------------------+
+---+---+--------------+--------------+--------------+--------------+--------------+--------------+----------------------+

